In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW17OT-MakeConcordance-Esther.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""


import os
import pickle
import string

from BibleWordList2 import *

BUILDAWIP = 1
BUILDNWIV = 1
BUILDPNIP = 1
BUILDNWIC = 1
BUILDNLIW = 1
BUILDBPIP = 1

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0





Esther01 = [
['Esther','Est','01','001','1',"Now it came to pass in the days of Ahasuerus, (this \emph{is} Ahasuerus which reigned, from India even unto Ethiopia, \emph{over} an hundred and seven and twenty provinces:)"],
['Esther','Est','01','002','2',"\emph{That} in those days, when the king Ahasuerus sat on the throne of his kingdom, which \emph{was} in Shushan the palace,"],
['Esther','Est','01','003','3',"In the third year of his reign, he made a feast unto all his princes and his servants; the power of Persia and Media, the nobles and princes of the provinces, \emph{being} before him:"],
['Esther','Est','01','004','4',"When he shewed the riches of his glorious kingdom and the honour of his excellent majesty many days, \emph{even} an hundred and fourscore days."],
['Esther','Est','01','005','5',"And when these days were expired, the king made a feast unto all the people that were present in Shushan the palace, both unto great and small, seven days, in the court of the garden of the king's palace;"],
['Esther','Est','01','006','6',"\emph{Where} \emph{were} white, green, and blue, \emph{hangings}, fastened with cords of fine linen and purple to silver rings and pillars of marble: the beds \emph{were} \emph{of} gold and silver, upon a pavement of red, and blue, and white, and black, marble."],
['Esther','Est','01','007','7',"And they gave \emph{them} drink in vessels of gold, (the vessels being diverse one from another,) and royal wine in abundance, according to the state of the king."],
['Esther','Est','01','008','8',"And the drinking \emph{was} according to the law; none did compel: for so the king had appointed to all the officers of his house, that they should do according to every man's pleasure."],
['Esther','Est','01','009','9',"Also Vashti the queen made a feast for the women \emph{in} the royal house which \emph{belonged} to king Ahasuerus."],
['Esther','Est','01','010','10',"On the seventh day, when the heart of the king was merry with wine, he commanded Mehuman, Biztha, Harbona, Bigtha, and Abagtha, Zethar, and Carcas, the seven chamberlains that served in the presence of Ahasuerus the king,"],
['Esther','Est','01','011','11',"To bring Vashti the queen before the king with the crown royal, to shew the people and the princes her beauty: for she \emph{was} fair to look on."],
['Esther','Est','01','012','12',"But the queen Vashti refused to come at the king's commandment by \emph{his} chamberlains: therefore was the king very wroth, and his anger burned in him."],
['Esther','Est','01','013','13',"Then the king said to the wise men, which knew the times, (for so \emph{was} the king's manner toward all that knew law and judgment:"],
['Esther','Est','01','014','14',"And the next unto him \emph{was} Carshena, Shethar, Admatha, Tarshish, Meres, Marsena, \emph{and} Memucan, the seven princes of Persia and Media, which saw the king's face, \emph{and} which sat the first in the kingdom;)"],
['Esther','Est','01','015','15',"What shall we do unto the queen Vashti according to law, because she hath not performed the commandment of the king Ahasuerus by the chamberlains?"],
['Esther','Est','01','016','16',"And Memucan answered before the king and the princes, Vashti the queen hath not done wrong to the king only, but also to all the princes, and to all the people that \emph{are} in all the provinces of the king Ahasuerus."],
['Esther','Est','01','017','17',"For \emph{this} deed of the queen shall come abroad unto all women, so that they shall despise their husbands in their eyes, when it shall be reported, The king Ahasuerus commanded Vashti the queen to be brought in before him, but she came not."],
['Esther','Est','01','018','18',"\emph{Likewise} shall the ladies of Persia and Media say this day unto all the king's princes, which have heard of the deed of the queen. Thus \emph{shall} \emph{there} \emph{arise} too much contempt and wrath."],
['Esther','Est','01','019','19',"If it please the king, let there go a royal commandment from him, and let it be written among the laws of the Persians and the Medes, that it be not altered, That Vashti come no more before king Ahasuerus; and let the king give her royal estate unto another that is better than she."],
['Esther','Est','01','020','20',"And when the king's decree which he shall make shall be published throughout all his empire, (for it is great,) all the wives shall give to their husbands honour, both to great and small."],
['Esther','Est','01','021','21',"And the saying pleased the king and the princes; and the king did according to the word of Memucan:"],
['Esther','Est','01','022','22',"For he sent letters into all the king's provinces, into every province according to the writing thereof, and to every people after their language, that every man should bear rule in his own house, and that \emph{it} should be published according to the language of every people."]
]

Esther02 = [
['Esther','Est','02','001','1',"After these things, when the wrath of king Ahasuerus was appeased, he remembered Vashti, and what she had done, and what was decreed against her."],
['Esther','Est','02','002','2',"Then said the king's servants that ministered unto him, Let there be fair young virgins sought for the king:"],
['Esther','Est','02','003','3',"And let the king appoint officers in all the provinces of his kingdom, that they may gather together all the fair young virgins unto Shushan the palace, to the house of the women, unto the custody of Hege the king's chamberlain, keeper of the women; and let their things for purification be given \emph{them}:"],
['Esther','Est','02','004','4',"And let the maiden which pleaseth the king be queen instead of Vashti. And the thing pleased the king; and he did so."],
['Esther','Est','02','005','5',"\emph{Now} in Shushan the palace there was a certain Jew, whose name \emph{was} Mordecai, the son of Jair, the son of Shimei, the son of Kish, a Benjamite;"],
['Esther','Est','02','006','6',"Who had been carried away from Jerusalem with the captivity which had been carried away with Jeconiah king of Judah, whom Nebuchadnezzar the king of Babylon had carried away."],
['Esther','Est','02','007','7',"And he brought up Hadassah, that \emph{is}, Esther, his uncle's daughter: for she had neither father nor mother, and the maid \emph{was} fair and beautiful; whom Mordecai, when her father and mother were dead, took for his own daughter."],
['Esther','Est','02','008','8',"So it came to pass, when the king's commandment and his decree was heard, and when many maidens were gathered together unto Shushan the palace, to the custody of Hegai, that Esther was brought also unto the king's house, to the custody of Hegai, keeper of the women."],
['Esther','Est','02','009','9',"And the maiden pleased him, and she obtained kindness of him; and he speedily gave her her things for purification, with such things as belonged to her, and seven maidens, \emph{which} \emph{were} meet to be given her, out of the king's house: and he preferred her and her maids unto the best \emph{place} of the house of the women."],
['Esther','Est','02','010','10',"Esther had not shewed her people nor her kindred: for Mordecai had charged her that she should not shew \emph{it}."],
['Esther','Est','02','011','11',"And Mordecai walked every day before the court of the women's house, to know how Esther did, and what should become of her."],
['Esther','Est','02','012','12',"Now when every maid's turn was come to go in to king Ahasuerus, after that she had been twelve months, according to the manner of the women, (for so were the days of their purifications accomplished, \emph{to} \emph{wit}, six months with oil of myrrh, and six months with sweet odours, and with \emph{other} things for the purifying of the women;)"],
['Esther','Est','02','013','13',"Then thus came \emph{every} maiden unto the king; whatsoever she desired was given her to go with her out of the house of the women unto the king's house."],
['Esther','Est','02','014','14',"In the evening she went, and on the morrow she returned into the second house of the women, to the custody of Shaashgaz, the king's chamberlain, which kept the concubines: she came in unto the king no more, except the king delighted in her, and that she were called by name."],
['Esther','Est','02','015','15',"Now when the turn of Esther, the daughter of Abihail the uncle of Mordecai, who had taken her for his daughter, was come to go in unto the king, she required nothing but what Hegai the king's chamberlain, the keeper of the women, appointed. And Esther obtained favour in the sight of all them that looked upon her."],
['Esther','Est','02','016','16',"So Esther was taken unto king Ahasuerus into his house royal in the tenth month, which \emph{is} the month Tebeth, in the seventh year of his reign."],
['Esther','Est','02','017','17',"And the king loved Esther above all the women, and she obtained grace and favour in his sight more than all the virgins; so that he set the royal crown upon her head, and made her queen instead of Vashti."],
['Esther','Est','02','018','18',"Then the king made a great feast unto all his princes and his servants, \emph{even} Esther's feast; and he made a release to the provinces, and gave gifts, according to the state of the king."],
['Esther','Est','02','019','19',"And when the virgins were gathered together the second time, then Mordecai sat in the king's gate."],
['Esther','Est','02','020','20',"Esther had not \emph{yet} shewed her kindred nor her people; as Mordecai had charged her: for Esther did the commandment of Mordecai, like as when she was brought up with him."],
['Esther','Est','02','021','21',"In those days, while Mordecai sat in the king's gate, two of the king's chamberlains, Bigthan and Teresh, of those which kept the door, were wroth, and sought to lay hand on the king Ahasuerus."],
['Esther','Est','02','022','22',"And the thing was known to Mordecai, who told \emph{it} unto Esther the queen; and Esther certified the king \emph{thereof} in Mordecai's name."],
['Esther','Est','02','023','23',"And when inquisition was made of the matter, it was found out; therefore they were both hanged on a tree: and it was written in the book of the chronicles before the king."]
]

Esther03 = [
['Esther','Est','03','001','1',"After these things did king Ahasuerus promote Haman the son of Hammedatha the Agagite, and advanced him, and set his seat above all the princes that \emph{were} with him."],
['Esther','Est','03','002','2',"And all the king's servants, that \emph{were} in the king's gate, bowed, and reverenced Haman: for the king had so commanded concerning him. But Mordecai bowed not, nor did \emph{him} reverence."],
['Esther','Est','03','003','3',"Then the king's servants, which \emph{were} in the king's gate, said unto Mordecai, Why transgressest thou the king's commandment?"],
['Esther','Est','03','004','4',"Now it came to pass, when they spake daily unto him, and he hearkened not unto them, that they told Haman, to see whether Mordecai's matters would stand: for he had told them that he \emph{was} a Jew."],
['Esther','Est','03','005','5',"And when Haman saw that Mordecai bowed not, nor did him reverence, then was Haman full of wrath."],
['Esther','Est','03','006','6',"And he thought scorn to lay hands on Mordecai alone; for they had shewed him the people of Mordecai: wherefore Haman sought to destroy all the Jews that \emph{were} throughout the whole kingdom of Ahasuerus, \emph{even} the people of Mordecai."],
['Esther','Est','03','007','7',"In the first month, that \emph{is}, the month Nisan, in the twelfth year of king Ahasuerus, they cast Pur, that \emph{is}, the lot, before Haman from day to day, and from month to month, \emph{to} the twelfth \emph{month}, that \emph{is}, the month Adar."],
['Esther','Est','03','008','8',"And Haman said unto king Ahasuerus, There is a certain people scattered abroad and dispersed among the people in all the provinces of thy kingdom; and their laws \emph{are} diverse from all people; neither keep they the king's laws: therefore it \emph{is} not for the king's profit to suffer them."],
['Esther','Est','03','009','9',"If it please the king, let it be written that they may be destroyed: and I will pay ten thousand talents of silver to the hands of those that have the charge of the business, to bring \emph{it} into the king's treasuries."],
['Esther','Est','03','010','10',"And the king took his ring from his hand, and gave it unto Haman the son of Hammedatha the Agagite, the Jews' enemy."],
['Esther','Est','03','011','11',"And the king said unto Haman, The silver \emph{is} given to thee, the people also, to do with them as it seemeth good to thee."],
['Esther','Est','03','012','12',"Then were the king's scribes called on the thirteenth day of the first month, and there was written according to all that Haman had commanded unto the king's lieutenants, and to the governors that \emph{were} over every province, and to the rulers of every people of every province according to the writing thereof, and \emph{to} every people after their language; in the name of king Ahasuerus was it written, and sealed with the king's ring."],
['Esther','Est','03','013','13',"And the letters were sent by posts into all the king's provinces, to destroy, to kill, and to cause to perish, all Jews, both young and old, little children and women, in one day, \emph{even} upon the thirteenth \emph{day} of the twelfth month, which is the month Adar, and \emph{to} \emph{take} the spoil of them for a prey."],
['Esther','Est','03','014','14',"The copy of the writing for a commandment to be given in every province was published unto all people, that they should be ready against that day."],
['Esther','Est','03','015','15',"The posts went out, being hastened by the king's commandment, and the decree was given in Shushan the palace. And the king and Haman sat down to drink; but the city Shushan was perplexed."]
]

Esther04 = [
['Esther','Est','04','001','1',"When Mordecai perceived all that was done, Mordecai rent his clothes, and put on sackcloth with ashes, and went out into the midst of the city, and cried with a loud and a bitter cry;"],
['Esther','Est','04','002','2',"And came even before the king's gate: for none \emph{might} enter into the king's gate clothed with sackcloth."],
['Esther','Est','04','003','3',"And in every province, whithersoever the king's commandment and his decree came, \emph{there} \emph{was} great mourning among the Jews, and fasting, and weeping, and wailing; and many lay in sackcloth and ashes."],
['Esther','Est','04','004','4',"So Esther's maids and her chamberlains came and told \emph{it} her. Then was the queen exceedingly grieved; and she sent raiment to clothe Mordecai, and to take away his sackcloth from him: but he received \emph{it} not."],
['Esther','Est','04','005','5',"Then called Esther for Hatach, \emph{one} of the king's chamberlains, whom he had appointed to attend upon her, and gave him a commandment to Mordecai, to know what it \emph{was}, and why it \emph{was}."],
['Esther','Est','04','006','6',"So Hatach went forth to Mordecai unto the street of the city, which \emph{was} before the king's gate."],
['Esther','Est','04','007','7',"And Mordecai told him of all that had happened unto him, and of the sum of the money that Haman had promised to pay to the king's treasuries for the Jews, to destroy them."],
['Esther','Est','04','008','8',"Also he gave him the copy of the writing of the decree that was given at Shushan to destroy them, to shew \emph{it} unto Esther, and to declare \emph{it} unto her, and to charge her that she should go in unto the king, to make supplication unto him, and to make request before him for her people."],
['Esther','Est','04','009','9',"And Hatach came and told Esther the words of Mordecai."],
['Esther','Est','04','010','10',"Again Esther spake unto Hatach, and gave him commandment unto Mordecai;"],
['Esther','Est','04','011','11',"All the king's servants, and the people of the king's provinces, do know, that whosoever, whether man or woman, shall come unto the king into the inner court, who is not called, \emph{there} \emph{is} one law of his to put \emph{him} to death, except such to whom the king shall hold out the golden sceptre, that he may live: but I have not been called to come in unto the king these thirty days."],
['Esther','Est','04','012','12',"And they told to Mordecai Esther's words."],
['Esther','Est','04','013','13',"Then Mordecai commanded to answer Esther, Think not with thyself that thou shalt escape in the king's house, more than all the Jews."],
['Esther','Est','04','014','14',"For if thou altogether holdest thy peace at this time, \emph{then} shall there enlargement and deliverance arise to the Jews from another place; but thou and thy father's house shall be destroyed: and who knoweth whether thou art come to the kingdom for \emph{such} a time as this?"],
['Esther','Est','04','015','15',"Then Esther bade \emph{them} return Mordecai \emph{this} \emph{answer},"],
['Esther','Est','04','016','16',"Go, gather together all the Jews that are present in Shushan, and fast ye for me, and neither eat nor drink three days, night or day: I also and my maidens will fast likewise; and so will I go in unto the king, which \emph{is} not according to the law: and if I perish, I perish."],
['Esther','Est','04','017','17',"So Mordecai went his way, and did according to all that Esther had commanded him."]
]

Esther05 = [
['Esther','Est','05','001','1',"Now it came to pass on the third day, that Esther put on \emph{her} royal \emph{apparel}, and stood in the inner court of the king's house, over against the king's house: and the king sat upon his royal throne in the royal house, over against the gate of the house."],
['Esther','Est','05','002','2',"And it was so, when the king saw Esther the queen standing in the court, \emph{that} she obtained favour in his sight: and the king held out to Esther the golden sceptre that \emph{was} in his hand. So Esther drew near, and touched the top of the sceptre."],
['Esther','Est','05','003','3',"Then said the king unto her, What wilt thou, queen Esther? and what \emph{is} thy request? it shall be even given thee to the half of the kingdom."],
['Esther','Est','05','004','4',"And Esther answered, If \emph{it} \emph{seem} good unto the king, let the king and Haman come this day unto the banquet that I have prepared for him."],
['Esther','Est','05','005','5',"Then the king said, Cause Haman to make haste, that he may do as Esther hath said. So the king and Haman came to the banquet that Esther had prepared."],
['Esther','Est','05','006','6',"And the king said unto Esther at the banquet of wine, What \emph{is} thy petition? and it shall be granted thee: and what \emph{is} thy request? even to the half of the kingdom it shall be performed."],
['Esther','Est','05','007','7',"Then answered Esther, and said, My petition and my request \emph{is};"],
['Esther','Est','05','008','8',"If I have found favour in the sight of the king, and if it please the king to grant my petition, and to perform my request, let the king and Haman come to the banquet that I shall prepare for them, and I will do to morrow as the king hath said."],
['Esther','Est','05','009','9',"Then went Haman forth that day joyful and with a glad heart: but when Haman saw Mordecai in the king's gate, that he stood not up, nor moved for him, he was full of indignation against Mordecai."],
['Esther','Est','05','010','10',"Nevertheless Haman refrained himself: and when he came home, he sent and called for his friends, and Zeresh his wife."],
['Esther','Est','05','011','11',"And Haman told them of the glory of his riches, and the multitude of his children, and all \emph{the} \emph{things} wherein the king had promoted him, and how he had advanced him above the princes and servants of the king."],
['Esther','Est','05','012','12',"Haman said moreover, Yea, Esther the queen did let no man come in with the king unto the banquet that she had prepared but myself; and to morrow am I invited unto her also with the king."],
['Esther','Est','05','013','13',"Yet all this availeth me nothing, so long as I see Mordecai the Jew sitting at the king's gate."],
['Esther','Est','05','014','14',"Then said Zeresh his wife and all his friends unto him, Let a gallows be made of fifty cubits high, and to morrow speak thou unto the king that Mordecai may be hanged thereon: then go thou in merrily with the king unto the banquet. And the thing pleased Haman; and he caused the gallows to be made."]
]

Esther06 = [
['Esther','Est','06','001','1',"On that night could not the king sleep, and he commanded to bring the book of records of the chronicles; and they were read before the king."],
['Esther','Est','06','002','2',"And it was found written, that Mordecai had told of Bigthana and Teresh, two of the king's chamberlains, the keepers of the door, who sought to lay hand on the king Ahasuerus."],
['Esther','Est','06','003','3',"And the king said, What honour and dignity hath been done to Mordecai for this? Then said the king's servants that ministered unto him, There is nothing done for him."],
['Esther','Est','06','004','4',"And the king said, Who \emph{is} in the court? Now Haman was come into the outward court of the king's house, to speak unto the king to hang Mordecai on the gallows that he had prepared for him."],
['Esther','Est','06','005','5',"And the king's servants said unto him, Behold, Haman standeth in the court. And the king said, Let him come in."],
['Esther','Est','06','006','6',"So Haman came in. And the king said unto him, What shall be done unto the man whom the king delighteth to honour? Now Haman thought in his heart, To whom would the king delight to do honour more than to myself?"],
['Esther','Est','06','007','7',"And Haman answered the king, For the man whom the king delighteth to honour,"],
['Esther','Est','06','008','8',"Let the royal apparel be brought which the king \emph{useth} to wear, and the horse that the king rideth upon, and the crown royal which is set upon his head:"],
['Esther','Est','06','009','9',"And let this apparel and horse be delivered to the hand of one of the king's most noble princes, that they may array the man \emph{withal} whom the king delighteth to honour, and bring him on horseback through the street of the city, and proclaim before him, Thus shall it be done to the man whom the king delighteth to honour."],
['Esther','Est','06','010','10',"Then the king said to Haman, Make haste, \emph{and} take the apparel and the horse, as thou hast said, and do even so to Mordecai the Jew, that sitteth at the king's gate: let nothing fail of all that thou hast spoken."],
['Esther','Est','06','011','11',"Then took Haman the apparel and the horse, and arrayed Mordecai, and brought him on horseback through the street of the city, and proclaimed before him, Thus shall it be done unto the man whom the king delighteth to honour."],
['Esther','Est','06','012','12',"And Mordecai came again to the king's gate. But Haman hasted to his house mourning, and having his head covered."],
['Esther','Est','06','013','13',"And Haman told Zeresh his wife and all his friends every \emph{thing} that had befallen him. Then said his wise men and Zeresh his wife unto him, If Mordecai \emph{be} of the seed of the Jews, before whom thou hast begun to fall, thou shalt not prevail against him, but shalt surely fall before him."],
['Esther','Est','06','014','14',"And while they \emph{were} yet talking with him, came the king's chamberlains, and hasted to bring Haman unto the banquet that Esther had prepared."]
]

Esther07 = [
['Esther','Est','07','001','1',"So the king and Haman came to banquet with Esther the queen."],
['Esther','Est','07','002','2',"And the king said again unto Esther on the second day at the banquet of wine, What \emph{is} thy petition, queen Esther? and it shall be granted thee: and what \emph{is} thy request? and it shall be performed, \emph{even} to the half of the kingdom."],
['Esther','Est','07','003','3',"Then Esther the queen answered and said, If I have found favour in thy sight, O king, and if it please the king, let my life be given me at my petition, and my people at my request:"],
['Esther','Est','07','004','4',"For we are sold, I and my people, to be destroyed, to be slain, and to perish. But if we had been sold for bondmen and bondwomen, I had held my tongue, although the enemy could not countervail the king's damage."],
['Esther','Est','07','005','5',"Then the king Ahasuerus answered and said unto Esther the queen, Who is he, and where is he, that durst presume in his heart to do so?"],
['Esther','Est','07','006','6',"And Esther said, The adversary and enemy \emph{is} this wicked Haman. Then Haman was afraid before the king and the queen."],
['Esther','Est','07','007','7',"And the king arising from the banquet of wine in his wrath \emph{went} into the palace garden: and Haman stood up to make request for his life to Esther the queen; for he saw that there was evil determined against him by the king."],
['Esther','Est','07','008','8',"Then the king returned out of the palace garden into the place of the banquet of wine; and Haman was fallen upon the bed whereon Esther \emph{was}. Then said the king, Will he force the queen also before me in the house? As the word went out of the king's mouth, they covered Haman's face."],
['Esther','Est','07','009','9',"And Harbonah, one of the chamberlains, said before the king, Behold also, the gallows fifty cubits high, which Haman had made for Mordecai, who had spoken good for the king, standeth in the house of Haman. Then the king said, Hang him thereon."],
['Esther','Est','07','010','10',"So they hanged Haman on the gallows that he had prepared for Mordecai. Then was the king's wrath pacified."]
]

Esther08 = [
['Esther','Est','08','001','1',"On that day did the king Ahasuerus give the house of Haman the Jews' enemy unto Esther the queen. And Mordecai came before the king; for Esther had told what he \emph{was} unto her."],
['Esther','Est','08','002','2',"And the king took off his ring, which he had taken from Haman, and gave it unto Mordecai. And Esther set Mordecai over the house of Haman."],
['Esther','Est','08','003','3',"And Esther spake yet again before the king, and fell down at his feet, and besought him with tears to put away the mischief of Haman the Agagite, and his device that he had devised against the Jews."],
['Esther','Est','08','004','4',"Then the king held out the golden sceptre toward Esther. So Esther arose, and stood before the king,"],
['Esther','Est','08','005','5',"And said, If it please the king, and if I have found favour in his sight, and the thing \emph{seem} right before the king, and I \emph{be} pleasing in his eyes, let it be written to reverse the letters devised by Haman the son of Hammedatha the Agagite, which he wrote to destroy the Jews which \emph{are} in all the king's provinces:"],
['Esther','Est','08','006','6',"For how can I endure to see the evil that shall come unto my people? or how can I endure to see the destruction of my kindred?"],
['Esther','Est','08','007','7',"Then the king Ahasuerus said unto Esther the queen and to Mordecai the Jew, Behold, I have given Esther the house of Haman, and him they have hanged upon the gallows, because he laid his hand upon the Jews."],
['Esther','Est','08','008','8',"Write ye also for the Jews, as it liketh you, in the king's name, and seal \emph{it} with the king's ring: for the writing which is written in the king's name, and sealed with the king's ring, may no man reverse."],
['Esther','Est','08','009','9',"Then were the king's scribes called at that time in the third month, that \emph{is}, the month Sivan, on the three and twentieth \emph{day} thereof; and it was written according to all that Mordecai commanded unto the Jews, and to the lieutenants, and the deputies and rulers of the provinces which \emph{are} from India unto Ethiopia, an hundred twenty and seven provinces, unto every province according to the writing thereof, and unto every people after their language, and to the Jews according to their writing, and according to their language."],
['Esther','Est','08','010','10',"And he wrote in the king Ahasuerus' name, and sealed \emph{it} with the king's ring, and sent letters by posts on horseback, \emph{and} riders on mules, camels, \emph{and} young dromedaries:"],
['Esther','Est','08','011','11',"Wherein the king granted the Jews which \emph{were} in every city to gather themselves together, and to stand for their life, to destroy, to slay, and to cause to perish, all the power of the people and province that would assault them, \emph{both} little ones and women, and \emph{to} \emph{take} the spoil of them for a prey,"],
['Esther','Est','08','012','12',"Upon one day in all the provinces of king Ahasuerus, \emph{namely}, upon the thirteenth \emph{day} of the twelfth month, which \emph{is} the month Adar."],
['Esther','Est','08','013','13',"The copy of the writing for a commandment to be given in every province \emph{was} published unto all people, and that the Jews should be ready against that day to avenge themselves on their enemies."],
['Esther','Est','08','014','14',"\emph{So} the posts that rode upon mules \emph{and} camels went out, being hastened and pressed on by the king's commandment. And the decree was given at Shushan the palace."],
['Esther','Est','08','015','15',"And Mordecai went out from the presence of the king in royal apparel of blue and white, and with a great crown of gold, and with a garment of fine linen and purple: and the city of Shushan rejoiced and was glad."],
['Esther','Est','08','016','16',"The Jews had light, and gladness, and joy, and honour."],
['Esther','Est','08','017','17',"And in every province, and in every city, whithersoever the king's commandment and his decree came, the Jews had joy and gladness, a feast and a good day. And many of the people of the land became Jews; for the fear of the Jews fell upon them."]
]

Esther09 = [
['Esther','Est','09','001','1',"Now in the twelfth month, that \emph{is}, the month Adar, on the thirteenth day of the same, when the king's commandment and his decree drew near to be put in execution, in the day that the enemies of the Jews hoped to have power over them, (though it was turned to the contrary, that the Jews had rule over them that hated them;)"],
['Esther','Est','09','002','2',"The Jews gathered themselves together in their cities throughout all the provinces of the king Ahasuerus, to lay hand on such as sought their hurt: and no man could withstand them; for the fear of them fell upon all people."],
['Esther','Est','09','003','3',"And all the rulers of the provinces, and the lieutenants, and the deputies, and officers of the king, helped the Jews; because the fear of Mordecai fell upon them."],
['Esther','Est','09','004','4',"For Mordecai \emph{was} great in the king's house, and his fame went out throughout all the provinces: for this man Mordecai waxed greater and greater."],
['Esther','Est','09','005','5',"Thus the Jews smote all their enemies with the stroke of the sword, and slaughter, and destruction, and did what they would unto those that hated them."],
['Esther','Est','09','006','6',"And in Shushan the palace the Jews slew and destroyed five hundred men."],
['Esther','Est','09','007','7',"And Parshandatha, and Dalphon, and Aspatha,"],
['Esther','Est','09','008','8',"And Poratha, and Adalia, and Aridatha,"],
['Esther','Est','09','009','9',"And Parmashta, and Arisai, and Aridai, and Vajezatha,"],
['Esther','Est','09','010','10',"The ten sons of Haman the son of Hammedatha, the enemy of the Jews, slew they; but on the spoil laid they not their hand."],
['Esther','Est','09','011','11',"On that day the number of those that were slain in Shushan the palace was brought before the king."],
['Esther','Est','09','012','12',"And the king said unto Esther the queen, The Jews have slain and destroyed five hundred men in Shushan the palace, and the ten sons of Haman; what have they done in the rest of the king's provinces? now what \emph{is} thy petition? and it shall be granted thee: or what \emph{is} thy request further? and it shall be done."],
['Esther','Est','09','013','13',"Then said Esther, If it please the king, let it be granted to the Jews which \emph{are} in Shushan to do to morrow also according unto this day's decree, and let Haman's ten sons be hanged upon the gallows."],
['Esther','Est','09','014','14',"And the king commanded it so to be done: and the decree was given at Shushan; and they hanged Haman's ten sons."],
['Esther','Est','09','015','15',"For the Jews that \emph{were} in Shushan gathered themselves together on the fourteenth day also of the month Adar, and slew three hundred men at Shushan; but on the prey they laid not their hand."],
['Esther','Est','09','016','16',"But the other Jews that \emph{were} in the king's provinces gathered themselves together, and stood for their lives, and had rest from their enemies, and slew of their foes seventy and five thousand, but they laid not their hands on the prey,"],
['Esther','Est','09','017','17',"On the thirteenth day of the month Adar; and on the fourteenth day of the same rested they, and made it a day of feasting and gladness."],
['Esther','Est','09','018','18',"But the Jews that \emph{were} at Shushan assembled together on the thirteenth \emph{day} thereof, and on the fourteenth thereof; and on the fifteenth \emph{day} of the same they rested, and made it a day of feasting and gladness."],
['Esther','Est','09','019','19',"Therefore the Jews of the villages, that dwelt in the unwalled towns, made the fourteenth day of the month Adar \emph{a} \emph{day} \emph{of} gladness and feasting, and a good day, and of sending portions one to another."],
['Esther','Est','09','020','20',"And Mordecai wrote these things, and sent letters unto all the Jews that \emph{were} in all the provinces of the king Ahasuerus, \emph{both} nigh and far,"],
['Esther','Est','09','021','21',"To stablish \emph{this} among them, that they should keep the fourteenth day of the month Adar, and the fifteenth day of the same, yearly,"],
['Esther','Est','09','022','22',"As the days wherein the Jews rested from their enemies, and the month which was turned unto them from sorrow to joy, and from mourning into a good day: that they should make them days of feasting and joy, and of sending portions one to another, and gifts to the poor."],
['Esther','Est','09','023','23',"And the Jews undertook to do as they had begun, and as Mordecai had written unto them;"],
['Esther','Est','09','024','24',"Because Haman the son of Hammedatha, the Agagite, the enemy of all the Jews, had devised against the Jews to destroy them, and had cast Pur, that \emph{is}, the lot, to consume them, and to destroy them;"],
['Esther','Est','09','025','25',"But when \emph{Esther} came before the king, he commanded by letters that his wicked device, which he devised against the Jews, should return upon his own head, and that he and his sons should be hanged on the gallows."],
['Esther','Est','09','026','26',"Wherefore they called these days Purim after the name of Pur. Therefore for all the words of this letter, and \emph{of} \emph{that} which they had seen concerning this matter, and which had come unto them,"],
['Esther','Est','09','027','27',"The Jews ordained, and took upon them, and upon their seed, and upon all such as joined themselves unto them, so as it should not fail, that they would keep these two days according to their writing, and according to their \emph{appointed} time every year;"],
['Esther','Est','09','028','28',"And \emph{that} these days \emph{should} \emph{be} remembered and kept throughout every generation, every family, every province, and every city; and \emph{that} these days of Purim should not fail from among the Jews, nor the memorial of them perish from their seed."],
['Esther','Est','09','029','29',"Then Esther the queen, the daughter of Abihail, and Mordecai the Jew, wrote with all authority, to confirm this second letter of Purim."],
['Esther','Est','09','030','30',"And he sent the letters unto all the Jews, to the hundred twenty and seven provinces of the kingdom of Ahasuerus, \emph{with} words of peace and truth,"],
['Esther','Est','09','031','31',"To confirm these days of Purim in their times \emph{appointed}, according as Mordecai the Jew and Esther the queen had enjoined them, and as they had decreed for themselves and for their seed, the matters of the fastings and their cry."],
['Esther','Est','09','032','32',"And the decree of Esther confirmed these matters of Purim; and it was written in the book."]
]

Esther10 = [
['Esther','Est','10','001','1',"And the king Ahasuerus laid a tribute upon the land, and \emph{upon} the isles of the sea."],
['Esther','Est','10','002','2',"And all the acts of his power and of his might, and the declaration of the greatness of Mordecai, whereunto the king advanced him, \emph{are} they not written in the book of the chronicles of the kings of Media and Persia?"],
['Esther','Est','10','003','3',"For Mordecai the Jew \emph{was} next unto king Ahasuerus, and great among the Jews, and accepted of the multitude of his brethren, seeking the wealth of his people, and speaking peace to all his seed."],
]





def main(): 
    ProcessVerses('Esther10',Esther10)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{And the king Ahasuerus laid a tribute upon the land, and \emph{upon} the isles of the sea.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And all the acts of his power and of his might, and the declaration of the greatness of Mordecai, whereunto the king advanced him, \emph{are} they not written in the book of the chronicles of the kings of Media and Persia?}
[3] \textcolor[cmyk]{0.99998,1,0,0}{For Mordecai the Jew \emph{was} next unto king Ahasuerus, and great among the Jews, and accepted of the multitude of his brethren, seeking the wealth of his people, and speaking peace to all his seed.}


%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with indexed items
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cm